In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

from sklearn.model_selection import KFold
from scipy import sparse as sps

Tensorflow is not available


c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val, data_test= split_data2(data, 0.2, validation_percentage=0.1)
data_train_val = data_train + data_val

In [3]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train_val, verbose=False)
candidate_recommender.fit()

[I 2024-01-02 22:13:47,484] A new study created in memory with name: P3Beta


KeyboardInterrupt: 

In [ ]:
n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12633,NaN
12634,NaN
12635,NaN


In [ ]:
from tqdm import tqdm

cutoff = 50

for user_id in tqdm(range(n_users)):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe

100%|██████████| 12638/12638 [00:06<00:00, 2038.96it/s]


,ItemID
UserID,
0,"[22146, 18210, 19966, 15720, 2843, 15344, 3413..."
1,"[9728, 2548, 12583, 89, 83, 227, 5682, 812, 17..."
2,"[5136, 7300, 1424, 20101, 1427, 10027, 17099, ..."
3,"[8073, 227, 812, 2612, 21626, 17936, 819, 6138..."
4,"[20641, 12920, 17723, 5897, 9873, 14041, 18001..."
...,...
12633,"[2817, 7069, 2807, 2814, 2806, 2818, 2811, 280..."
12634,"[11498, 3943, 3949, 3955, 17082, 17076, 11259,..."
12635,"[3297, 18126, 2835, 16895, 4864, 14328, 19417,..."


In [ ]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

,ItemID
UserID,
0,22146
0,18210
0,19966
0,15720
0,2843
...,...
12637,688
12637,16322
12637,5005


In [ ]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_test)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})

correct_recommendations

,UserID,ItemID
0,0,8
1,0,13
2,0,20
3,0,35
4,1,10
...,...,...
95741,12637,7533
95742,12637,8797
95743,12637,10897
95744,12637,13074


In [ ]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)

training_dataframe

,UserID,ItemID,Label
0,0,22146,False
1,0,18210,False
2,0,19966,False
3,0,15720,False
4,0,2843,False
...,...,...,...
631895,12637,688,False
631896,12637,16322,False
631897,12637,5005,False
631898,12637,694,False


In [ ]:
topPop = TopPop(data_train_val)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train_val)
p3alpha.fit()

easer_study = optuna.create_study(
    study_name="Easer",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train_val)
easer.fit()

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train_val)
itemknncf.fit()


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

[I 2024-01-02 20:20:41,823] A new study created in memory with name: P3Alpha


TopPopRecommender: URM Detected 212 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 108 ( 0.5%) items with no interactions.
P3alphaRecommender: URM Detected 212 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 108 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 3039.63 column/sec. Elapsed time 7.31 sec


[I 2024-01-02 20:20:50,711] A new study created in memory with name: Easer


EASE_R_Recommender: URM Detected 212 ( 1.7%) users with no interactions.
EASE_R_Recommender: URM Detected 108 ( 0.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 5.07 min


[I 2024-01-02 20:26:22,749] A new study created in memory with name: ItemKNNCF


ItemKNNCFRecommender: URM Detected 212 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 108 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 7767.69 column/sec. Elapsed time 2.86 sec


In [ ]:
total_size = data_train.size + data_val.size + data_test.size
percentage_train = (data_train.size / total_size) * 100
percentage_val = (data_val.size / total_size) * 100
percentage_test = (data_test.size / total_size) * 100

print(f"Percentuale data_train: {percentage_train}%")
print(f"Percentuale data_val: {percentage_val}%")
print(f"Percentuale data_test: {percentage_test}%")

Percentuale data_train: 71.99987466839346%
Percentuale data_val: 8.000125331606542%
Percentuale data_test: 20.0%


In [ ]:
import scipy.sparse as sps
from tqdm import tqdm

training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

training_dataframe

100%|██████████| 12638/12638 [00:41<00:00, 305.18it/s]


,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF
0,0,22146,False,1.0,0.001263,0.002303,0.026965
1,0,18210,False,4.0,0.020833,0.001392,0.009352
2,0,19966,False,4.0,0.020833,0.001913,0.000000
3,0,15720,False,4.0,0.035714,0.001573,0.009725
4,0,2843,False,5.0,0.010638,0.003106,0.015204
...,...,...,...,...,...,...,...
631895,12637,688,False,174.0,0.013397,0.135440,0.890360
631896,12637,16322,False,4.0,0.004683,0.006868,0.049920
631897,12637,5005,False,37.0,0.024316,0.017416,0.198424
631898,12637,694,False,112.0,0.000000,0.103420,0.837656


In [ ]:
item_popularity = np.ediff1d(sps.csc_matrix(data_train_val).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train_val).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
training_dataframe

,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF,item_popularity,user_profile_len
0,0,22146,False,1.0,0.001263,0.002303,0.026965,1,40
1,0,18210,False,4.0,0.020833,0.001392,0.009352,4,40
2,0,19966,False,4.0,0.020833,0.001913,0.000000,4,40
3,0,15720,False,4.0,0.035714,0.001573,0.009725,4,40
4,0,2843,False,5.0,0.010638,0.003106,0.015204,5,40
...,...,...,...,...,...,...,...,...,...
631895,12637,688,False,174.0,0.013397,0.135440,0.890360,174,61
631896,12637,16322,False,4.0,0.004683,0.006868,0.049920,4,61
631897,12637,5005,False,37.0,0.024316,0.017416,0.198424,37,61
631898,12637,694,False,112.0,0.000000,0.103420,0.837656,112,61


In [ ]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns = ['index'], inplace=True)
training_dataframe

,ItemID,UserID,Label,TopPop,P3alpha,Easer,ItemKNNCF,item_popularity,user_profile_len
0,22146,0,False,1.0,0.001263,0.002303,0.026965,1,40
1,731,0,False,35.0,0.009256,0.022342,0.239549,35,40
2,2131,0,False,4.0,0.003906,0.000486,0.000000,4,40
3,16493,0,False,3.0,0.003546,-0.000071,0.000000,3,40
4,16847,0,False,16.0,0.011031,0.021394,0.206096,16,40
...,...,...,...,...,...,...,...,...,...
631895,13255,12637,False,7.0,0.013175,0.033991,0.358365,7,61
631896,9232,12637,False,3.0,0.006603,0.007133,0.048130,3,61
631897,16593,12637,False,8.0,0.012839,0.039575,0.387843,8,61
631898,19760,12637,False,3.0,0.001821,0.012163,0.045685,3,61


In [ ]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [ ]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-02 20:34:52,259] A new study created in memory with name: XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)


In [ ]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [ ]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        training_dataframe.drop(columns=["Label"]),
        training_dataframe["Label"],
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train_val, train_recs, XGB_model, training_dataframe, data_test)
    
    return ev_map

study.optimize(objective, n_trials=1000)

#Trial 6 finished with value: 0.12485196495596997 and parameters: {'objective': 'rank:ndcg', 'n_estimators': 336, 'learning_rate': 0.023887567768263025, 'reg_alpha': 0.01495634916569455, 'reg_lambda': 0.004873292863879341, 'max_depth': 4, 'max_leaves': 3, 'grow_policy': 'depthwise', 'booster': 'gbtree'}. Best is trial 6 with value: 0.12485196495596997.

[W 2024-01-02 22:13:23,366] Trial 16 failed with parameters: {'objective': 'rank:ndcg', 'n_estimators': 447, 'learning_rate': 0.059453630539483715, 'reg_alpha': 0.01584695949184005, 'reg_lambda': 0.014754540718805116, 'max_depth': 2, 'max_leaves': 5, 'grow_policy': 'depthwise', 'booster': 'dart'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Francesco1\AppData\Local\Temp\ipykernel_10820\3995153333.py", line 28, in objective
    XGB_model.fit(
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\xgboost\sklearn.py", line 2033, in fit
    self._Booster = train(
  File "c:\Users\Francesco1\miniconda3\envs

KeyboardInterrupt: 

In [ ]:
study.best_params

{'objective': 'rank:map',
 'n_estimators': 161,
 'learning_rate': 0.00013256002295690622,
 'reg_alpha': 0.008487026488935363,
 'reg_lambda': 0.0029756545632511457,
 'max_depth': 6,
 'max_leaves': 8,
 'grow_policy': 'lossguide',
 'booster': 'gbtree'}

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
    enable_categorical=True,
)
XGB_model.fit(
    train_recs,
    train_labs,
    group=groups,
    verbose=False
)

#XGB_model = XGBRanker(
#    objective='rank:ndcg', 
#    n_estimators=336, 
#    learning_rate=0.023887567768263025, 
#    reg_alpha=0.01495634916569455, 
#    reg_lambda=0.004873292863879341, 
#    max_depth=4, 
#    max_leaves=3, 
#    grow_policy= 'depthwise', 
#    booster='gbtree',
#    ndcg_exp_gain=False,
#    verbosity=0, # 2 if self.verbose else 0,
#    enable_categorical=True,
#)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None,
          device=None, early_stopping_rounds=None, enable_categorical=True,
          eval_metric=None, feature_types=None, gamma=None,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints=None, learning_rate=0.023887567768263025,
          max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
          max_delta_step=None, max_depth=4, max_leaves=3, min_child_weight=None,
          missing=nan, monotone_constraints=None, multi_strategy=None,
          n_estimators=336, n_jobs=None, ndcg_exp_gain=False,
          num_parallel_tree=None, ...)

In [ ]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)

TopPopRecommender: URM Detected 212 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 108 ( 0.5%) items with no interactions.
